In [12]:
!pip install -U -q pydrive2
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np
import imageio
# imageio.imwrite('filename.jpg', array)
# import scipy.ndimage as nd
import matplotlib.pyplot as plt
from google.colab import drive
import h5py
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import time
from keras import backend as K
# import scipy.ndimage as nd

import keras

from keras.models import Sequential,load_model
from keras import regularizers
from keras import initializers
from keras import metrics
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Convolution2D, MaxPooling2D, BatchNormalization, ReLU, LeakyReLU
from keras import layers
from keras import models
from keras.utils import to_categorical
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [6]:
drive.mount('/content/drive')
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# download TinyImageNet
! git clone https://github.com/seshuad/IMagenet


fatal: destination path 'IMagenet' already exists and is not an empty directory.


In [16]:
def get_id_dictionary():
    id_dict = {}
    for i, line in enumerate(open('IMagenet/tiny-imagenet-200/wnids.txt', 'r')):
        id_dict[line.replace('\n', '')] = i
    return id_dict

def get_class_to_id_dict():
    id_dict = get_id_dictionary()
    all_classes = {}
    result = {}
    for i, line in enumerate(open('IMagenet/tiny-imagenet-200/words.txt', 'r')):
        n_id, word = line.split('\t')[:2]
        all_classes[n_id] = word
    for key, value in id_dict.items():
        result[value] = (key, all_classes[key])

    return result

def get_data(id_dict):

    print('starting loading data')
    train_data, val_data, test_data = [], [], []
    train_labels, val_labels, test_labels = [], [], []
    t = time.time()
    for key, value in id_dict.items():
        train_data += [imageio.imread('IMagenet/tiny-imagenet-200/train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), mode='RGB') for i in range(450)]

        train_labels_ = np.array([[0]*200]*450)
        train_labels_[:, value] = 1
        train_labels += train_labels_.tolist()

        val_data += [imageio.imread('IMagenet/tiny-imagenet-200/train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), mode='RGB') for i in range(450, 500)]

        val_labels_ = np.array([[0]*200]*50)
        val_labels_[:, value] = 1
        val_labels += val_labels_.tolist()

    for line in open('IMagenet/tiny-imagenet-200/val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        test_data.append(imageio.imread('IMagenet/tiny-imagenet-200/val/images/{}'.format(img_name), mode='RGB'))

        test_labels_ = np.array([[0]*200])
        test_labels_[0, id_dict[class_id]] = 1
        test_labels += test_labels_.tolist()

    print('finished loading data, in {} seconds'.format(time.time() - t))

    return np.array(train_data), np.array(train_labels), np.array(val_data), np.array(val_labels), np.array(test_data), np.array(test_labels)

def make_augmented_train(id_dict):
    if not(os.path.isdir('drive/My Drive/tiny-imagenet-200-A')):
        print('making dir')
        os.mkdir('drive/My Drive/tiny-imagenet-200-A')
        os.mkdir('drive/My Drive/tiny-imagenet-200-A/train')
        os.mkdir('drive/My Drive/tiny-imagenet-200-A/test')

    crops = [((4, 60), (4, 60)),
             ((0, 56), (0, 56)),
             ((8, 64), (8, 64)),
             ((0, 56), (8, 64)),
             ((8, 64), (0, 56))]

    trans = [((4, 60), (4, 60)),
             ((8, 64), (8, 64)),
             ((0, 56), (0, 56)),
             ((8, 64), (0, 56)),
             ((0, 56), (8, 64))]

    print('starting creating augmeneted data')
    train_data, train_labels = [], []
    t = time.time()

    progress = 0
    for key, value in id_dict.items():
        print(progress)
        progress += 1
        if not(os.path.isdir('drive/My Drive/tiny-imagenet-200-A/train/{}'.format(key))):
            os.mkdir('drive/My Drive/tiny-imagenet-200-A/train/{}'.format(key))
            os.mkdir('drive/My Drive/tiny-imagenet-200-A/train/{}/images'.format(key))
        count = 0
        for crp, trs in zip(crops, trans):
            crop_x, crop_y = crp
            tran_x, tran_y = trs
            for mirror in [-1, 1]:
                for i in range(450):
                    img = imageio.imread('IMagenet/tiny-imagenet-200/train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), mode='RGB')[:, ::mirror][crop_x[0]:crop_x[1], crop_y[0]:crop_y[1]]
                    processed_img = np.zeros(shape = (64, 64, 3))
                    processed_img[tran_x[0]:tran_x[1], tran_y[0]:tran_y[1]] = img

                    imageio.imwrite('drive/My Drive/tiny-imagenet-200-A/train/{}/images/{}_{}.JPEG'.format(key, key, str(count)), processed_img)
                    count += 1

    print('finished creating augmeneted data, in {} seconds'.format(time.time() - t))

def shuffle_data(train_data, train_labels, val_data, val_labels):
    size = len(train_data)
    train_idx = np.arange(size)
    np.random.shuffle(train_idx)

    size = len(val_data)
    val_idx = np.arange(size)
    np.random.shuffle(val_idx)

    return train_data[train_idx], train_labels[train_idx], val_data[val_idx], val_labels[val_idx]

def alexnet(randomSeed = 0):
    tf.random.set_seed(randomSeed)
    model=models.Sequential([
    layers.Conv2D(filters=128, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(64,64,3)),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size=(3,3)),
    layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(1024,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1024,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(200)
    ])
    return model


train_data, train_labels, val_data, val_labels, test_data, test_labels = get_data(get_id_dictionary())
train_data, train_labels, val_data, val_labels = shuffle_data(train_data, train_labels, val_data, val_labels)


starting loading data


<ipython-input-16-2a78e894a689>:26: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  train_data += [imageio.imread('IMagenet/tiny-imagenet-200/train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), mode='RGB') for i in range(450)]
<ipython-input-16-2a78e894a689>:32: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  val_data += [imageio.imread('IMagenet/tiny-imagenet-200/train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), mode='RGB') for i in range(450, 500)]
<ipython-input-16-2a78e894a689>:40: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of

finished loading data, in 83.91088962554932 seconds


In [ ]:
model = alexnet()
custom_loss = keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=custom_loss, metrics=['accuracy'])
model.fit(train_data, train_labels, batch_size=128, epochs=30, validation_data=(val_data, val_labels))

Epoch 1/30
105/704 [===>..........................] - ETA: 13:21 - loss: 5.3528 - accuracy: 0.0089